In [1]:
import sys
sys.path.append("/home/oldrain123/IMBALANCED_CLASSIFICATION/MOMs")

In [5]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score
from imblearn.metrics import geometric_mean_score
from ctgan import CTGAN
from moms_generate import transform
from moms_losses import MMD_est_torch
import torch
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, ADASYN

from tqdm import tqdm 

In [6]:
# ----------------------------------------
# Experimental Parameters
# ----------------------------------------
SEED = 1203
random.seed(SEED)
np.random.seed(SEED)

MAJORITY_COUNT = 2000
MINORITY_COUNT = 200
N_TEST_PER_CLASS = 2000
# Classifiers to evaluate
CLASSIFIERS = {
    'SVM': SVC(probability=True, random_state=SEED),
    'RandomForest': RandomForestClassifier(n_estimators=100, random_state=SEED),
    'DecisionTree': DecisionTreeClassifier(random_state=SEED),
    'MLP': MLPClassifier(hidden_layer_sizes=(100,), max_iter=10000, random_state=SEED),
    'kNN': KNeighborsClassifier()
}

# Dimensions to evaluate
DIM_LIST = [50, 100, 200, 500, 1000]
TRIALS = 10

In [7]:
# ----------------------------------------
# Oversamplers to compare
# ----------------------------------------
oversamplers = {
    'Original': None,
    'SMOTE': SMOTE(random_state=SEED),
    'bSMOTE': BorderlineSMOTE(random_state=SEED),
    'ADASYN': ADASYN(random_state=SEED)
}

In [9]:
# ----------------------------------------
# Main Experiment Loop
# ----------------------------------------
records = []
for DIM in DIM_LIST:
    # Generate fixed test set for this dimension
    mu_maj = np.zeros(DIM)
    mu_min = np.ones(DIM) * 0.5
    cov = np.eye(DIM)

    X_test_maj = np.random.multivariate_normal(mu_maj, cov, size=N_TEST_PER_CLASS)
    X_test_min = np.random.multivariate_normal(mu_min, cov, size=N_TEST_PER_CLASS)
    X_test = np.vstack([X_test_maj, X_test_min])
    y_test = np.hstack([np.zeros(N_TEST_PER_CLASS), np.ones(N_TEST_PER_CLASS)])

    for trial in tqdm(range(TRIALS), desc=f"DIM={DIM}"):
        # Generate train set with fixed minority count
        X_maj = np.random.multivariate_normal(mu_maj, cov, size=MAJORITY_COUNT)
        X_min = np.random.multivariate_normal(mu_min, cov, size=MINORITY_COUNT)
        y_maj = np.zeros(MAJORITY_COUNT)
        y_min = np.ones(MINORITY_COUNT)

        X = np.vstack([X_maj, X_min])
        y = np.hstack([y_maj, y_min])

        X_train, _, y_train, _ = train_test_split(
            X, y, stratify=y, train_size=0.7, random_state=SEED + trial
        )

        # Evaluate each oversampling method
        for method, sampler in oversamplers.items():
            if sampler is not None:
                try:
                    X_res, y_res = sampler.fit_resample(X_train, y_train)
                except RuntimeError as e:
                    print(f"Warning: {method} failed with error '{e}'. Falling back to original training data.")
                    X_res, y_res = X_train, y_train
            else:
                X_res, y_res = X_train, y_train

            # Train and evaluate classifier
            for clf_name, clf in CLASSIFIERS.items():
                model = clf.fit(X_res, y_res)
                y_prob = model.predict_proba(X_test)[:, 1]
                y_pred = (y_prob >= 0.5).astype(int)

                records.append({
                    'dimension': DIM,
                    'trial': trial,
                    'method': method,
                    'classifier': clf_name,
                    'AUROC': roc_auc_score(y_test, y_prob),
                    'F1': f1_score(y_test, y_pred)
                })

DIM=1000:   0%|          | 0/10 [00:00<?, ?it/s]

DIM=1000:  30%|███       | 3/10 [03:36<08:31, 73.06s/it]

DIM=1000:  50%|█████     | 5/10 [06:10<06:18, 75.61s/it]

DIM=1000:  60%|██████    | 6/10 [07:16<04:49, 72.40s/it]

DIM=1000:  90%|█████████ | 9/10 [10:54<01:12, 72.86s/it]

DIM=1000: 100%|██████████| 10/10 [12:04<00:00, 72.48s/it]


In [14]:
# Compile results
results_df = pd.DataFrame(records)
results = results_df.groupby(['dimension', 'classifier', 'method']).agg({
    'AUROC': ['mean', 'std'],
    'F1': ['mean', 'std']
}).reset_index()

In [17]:
results.tail(30)

dimension    classifier    method     AUROC                      F1  \
                                          mean           std      mean   
70       500  RandomForest     SMOTE  0.999878  3.494286e-05  0.428012   
71       500  RandomForest    bSMOTE  0.997924  7.182985e-04  0.001199   
72       500           SVM    ADASYN  1.000000  0.000000e+00  1.000000   
73       500           SVM  Original  1.000000  0.000000e+00  1.000000   
74       500           SVM     SMOTE  1.000000  0.000000e+00  1.000000   
75       500           SVM    bSMOTE  1.000000  0.000000e+00  1.000000   
76       500           kNN    ADASYN  0.603875  3.582970e-02  0.675793   
77       500           kNN  Original  0.999975  7.907018e-05  0.998647   
78       500           kNN     SMOTE  0.618100  2.446233e-02  0.675238   
79       500           kNN    bSMOTE  0.963950  7.782034e-02  0.955457   
80      1000  DecisionTree    ADASYN  0.601850  5.977039e-03  0.405080   
81      1000  DecisionTree  Original  0.599500  5.756639e-03  0.399011   
82      1000  DecisionTree     SMOTE  0.642600  1.033347e-02  0.517914   
83      1000  DecisionTree    bSMOTE  0.599500  5.756639e-03  0.399011   
84      1000           MLP    ADASYN  1.000000  7.905694e-08  0.999326   
85      1000           MLP  Original  1.000000  7.905694e-08  0.998876   
86      1000           MLP     SMOTE  1.000000  0.000000e+00  0.999525   
87      1000           MLP    bSMOTE  1.000000  7.905694e-08  0.998876   
88      1000  RandomForest    ADASYN  0.999044  5.180026e-04  0.000300   
89      1000  RandomForest  Original  0.998655  4.035821e-04  0.000400   
90      1000  RandomForest     SMOTE  0.999978  1.258974e-05  0.386834   
91      1000  RandomForest    bSMOTE  0.998655  4.035821e-04  0.000400   
92      1000           SVM    ADASYN  1.000000  0.000000e+00  1.000000   
93      1000           SVM  Original  1.000000  0.000000e+00  1.000000   
94      1000           SVM     SMOTE  1.000000  0.000000e+00  1.000000   
95      1000           SVM    bSMOTE  1.000000  0.000000e+00  1.000000   
96      1000           kNN    ADASYN  0.784750  2.282409e-01  0.837508   
97      1000           kNN  Original  1.000000  0.000000e+00  1.000000   
98      1000           kNN     SMOTE  0.508375  2.131933e-03  0.666733   
99      1000           kNN    bSMOTE  1.000000  0.000000e+00  1.000000   

              
         std  
70  0.016538  
71  0.001228  
72  0.000000  
73  0.000000  
74  0.000000  
75  0.000000  
76  0.005293  
77  0.000955  
78  0.003715  
79  0.092294  
80  0.018066  
81  0.017880  
82  0.025120  
83  0.017880  
84  0.000391  
85  0.000317  
86  0.000219  
87  0.000317  
88  0.000483  
89  0.000699  
90  0.021636  
91  0.000699  
92  0.000000  
93  0.000000  
94  0.000000  
95  0.000000  
96  0.171359  
97  0.000000  
98  0.000057  
99  0.000000